# Imports

In [ ]:
import sys
sys.path.append('/home/rcendre/classification')
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="1";  

import itertools
import webbrowser
from pathlib import Path
import matplotlib.pyplot as plt
from numpy import array, logspace
from scipy.stats import randint as randint
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import davies_bouldin_score
from toolbox.classification.common import Folds, IO, Tools
from toolbox.classification.parameters import Dermatology, Settings
from toolbox.models.builtin import Applications
from toolbox.IO import dermatology
from toolbox.transforms.common import PredictorTransform
from toolbox.transforms.labels import OrderedEncoder
from toolbox.transforms.images import DistributionImageTransform, DWTImageTransform, FourierImageTransform, HaralickImageTransform, SpatialImageTransform
from toolbox.views.common import Views, ViewsTools
from toolbox.views.images import ImagesViews
from toolbox.classification.parameters import Dermatology, Settings, LocalParameters

# Parameters

In [ ]:
# Advanced parameters
use_unknown = False 
validation = 4
settings = Settings.get_default_dermatology()

# Microscopy

In [ ]:
inputs = Dermatology.images(modality='Microscopy', data_type='Full', use_unknown=True, location=Path().home()/'Data/Skin/Elisa.csv')

In [ ]:
# Transform groups
group_encoder = LabelEncoder().fit(array(inputs['ID_Patient'].tolist()))
Tools.transform(inputs, {'datum': 'ID_Patient'}, group_encoder, 'GroupEncode')
# Transform labels
lesion_encoder = OrderedEncoder().fit(['Benign', 'Malignant'])
Tools.transform(inputs, {'datum': 'Binary_Diagnosis'}, lesion_encoder, 'LesionEncode')

label_encoder = OrderedEncoder().fit(['Normal', 'Benign', 'Malignant'])
Tools.transform(inputs, {'datum': 'Label'}, label_encoder, 'LabelEncode')

malignant_encoder = OrderedEncoder().fit(['Rest', 'Malignant'])
Tools.transform(inputs, {'datum': 'Malignant'}, malignant_encoder, 'MalignantEncode')
# Make folds
Folds.build_group_folds(inputs, {'datum': 'Datum', 'label_encode': 'LabelEncode', 'group': 'GroupEncode'}, validation)

folds = Folds.export_group_folds(inputs, {'group': 'ID_Patient'})

In [ ]:
architectures = [('VGG16', {}), ('InceptionV3', {}), ('InceptionResNetV2', {}), ('ResNet', {})]
for architecture in architectures:
    architecture_name = architecture[0]
    architecture_additional = architecture[1]
    # Extracting
    method = Applications.get_transfer_learning(architecture=architecture_name, pooling='avg', batch_size=1, additional=architecture_additional)
    method.transform = method.predict_proba
    Tools.transform(inputs, {'datum': 'Datum'}, method, f'{architecture_name}Avg')

    # Extracting
    method = Applications.get_transfer_learning(architecture=architecture_name, pooling='max', batch_size=1, additional=architecture_additional)
    method.transform = method.predict_proba
    Tools.transform(inputs, {'datum': 'Datum'}, method, f'{architecture_name}Max')
    
    # Save    
    IO.save(inputs, 'Microscopy.pickle')

# Photography

In [ ]:
inputs = Dermatology.images(modality='Photography', data_type='Crop')

In [ ]:
# Transform groups
group_encoder = LabelEncoder().fit(array(inputs['ID_Patient'].tolist()))
Tools.transform(inputs, {'datum': 'ID_Patient'}, group_encoder, 'GroupEncode')
# Transform labels
lesion_encoder = OrderedEncoder().fit(['Benign', 'Malignant'])
Tools.transform(inputs, {'datum': 'Binary_Diagnosis'}, lesion_encoder, 'LesionEncode')
# Make folds
Folds.restore_group_folds(inputs, folds, {'group': 'ID_Patient'})

In [ ]:
architectures = [('VGG16', {}), ('InceptionV3', {}), ('InceptionResNetV2', {}), ('ResNet', {})]
for architecture in architectures:
    architecture_name = architecture[0]
    architecture_additional = architecture[1]
    # Extracting
    method = Applications.get_transfer_learning(architecture=architecture_name, pooling='avg', batch_size=1, additional=architecture_additional)
    method.transform = method.predict_proba
    Tools.transform(inputs, {'datum': 'Datum'}, method, f'{architecture_name}Avg')

    # Extracting
    method = Applications.get_transfer_learning(architecture=architecture_name, pooling='max', batch_size=1, additional=architecture_additional)
    method.transform = method.predict_proba
    Tools.transform(inputs, {'datum': 'Datum'}, method, f'{architecture_name}Max')
    
    # Save    
    IO.save(inputs, 'Photography.pickle')

# Dermoscopy

In [ ]:
inputs = Dermatology.images(modality='Dermoscopy')

In [ ]:
# Transform groups
group_encoder = LabelEncoder().fit(array(inputs['ID_Patient'].tolist()))
Tools.transform(inputs, {'datum': 'ID_Patient'}, group_encoder, 'GroupEncode')
# Transform labels
lesion_encoder = OrderedEncoder().fit(['Benign', 'Malignant'])
Tools.transform(inputs, {'datum': 'Binary_Diagnosis'}, lesion_encoder, 'LesionEncode')
# Make folds
Folds.restore_group_folds(inputs, folds, {'group': 'ID_Patient'})

In [ ]:
architectures = [('VGG16', {}), ('InceptionV3', {}), ('InceptionResNetV2', {}), ('ResNet', {})]
for architecture in architectures:
    architecture_name = architecture[0]
    architecture_additional = architecture[1]
    # Extracting
    method = Applications.get_transfer_learning(architecture=architecture_name, pooling='avg', batch_size=1, additional=architecture_additional)
    method.transform = method.predict_proba
    Tools.transform(inputs, {'datum': 'Datum'}, method, f'{architecture_name}Avg')

    # Extracting
    method = Applications.get_transfer_learning(architecture=architecture_name, pooling='max', batch_size=1, additional=architecture_additional)
    method.transform = method.predict_proba
    Tools.transform(inputs, {'datum': 'Datum'}, method, f'{architecture_name}Max')
    
    # Save    
    IO.save(inputs, 'Dermoscopy.pickle')